### Comparing the columns of the data structure defined inside schema and the MongoDB received Dataframe

In [ ]:
from sensor.utils.main_utils import read_yaml_file

schema_path ="D:\Industry Ready Projects\Sensor_Fault_Detection\config\schema.yaml"

schema_config = read_yaml_file(file_path=schema_path)

#len(schema_config['columns']) == len(df.columns)  # checking syntax

### Data drift test and generate Yaml report

In [1]:
import numpy as np
from scipy.stats import ks_2samp

In [2]:
d1 = np.arange(10)
d2 = np.arange(10)

ks_2samp(d1,d2) 

# Note : the p-value is 1 indicates there is no data drift between d1 and d2

KstestResult(statistic=0.0, pvalue=1.0)

In [9]:
d1 = np.random.uniform(-1,0,10)
d2 = np.random.lognormal(3,1,1000)

ks_2samp(d1,d2) 

# Note : we can see both datasets belongs to different distributions and hence p-value is zero.

KstestResult(statistic=1.0, pvalue=6.870526175538918e-24)

In [ ]:
traing_file_path = "/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/live_class/sensor-fault-detection/artifact/10_29_2022_12_29_11/data_ingestion/ingested/train.csv"
test_file_path = "/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/live_class/sensor-fault-detection/artifact/10_29_2022_12_29_11/data_ingestion/ingested/test.csv"
import pandas as pd
train_df = pd.read_csv(traing_file_path)
test_df = pd.read_csv(test_file_path)
train_df.columns

In [ ]:
# Now we will check in train and test datasets. 
# Write a fucntion which check wether there is a drift and return a report of more drifted columns with respective p-value.
# Note : data drift test can be conducted with catgeorical columns as well.
# Suppose we have defined or called train and test dfs 

def get_drift_report(base_df,current_df,threshold=0.6):  
    # only accept if atleast 50% of the distributions of train and test columns match
    report ={}

    for column in base_df.columns:
        d1 = base_df[column]
        d2 = current_df[column]
        is_same_dist = ks_2samp(d1,d2)
        if is_same_dist.pvalue<= threshold:
            is_found=False
        else:
            is_found=True
        report.update({column:{
            "p-value": float(is_same_dist.pvalue),
            "data drift status": is_found
            }})
    return report

drift_report = get_drift_report(train_df, test_df)

In [ ]:
# Generate yaml file of the data drift report
from sensor.utils.main_utils import write_yaml_file
import os

write_yaml_file(file_path=os.path.join(os.getcwd(),"report.yaml"),content=drift_report,replace=True)